## Feature Engineering

In [ ]:
import datetime as dt
from datetime import timedelta
import numpy as np
import pandas as pd
from data_wrangling import restrict_df_to_time_period
from data_wrangling import get_training_data_from_visits
from data_wrangling import fill_empty_values
from data_wrangling import create_features
from data_wrangling import CategoricalEncoder

In [ ]:
trainCSV = pd.read_csv('train_v2_flat.csv', sep=',', encoding='utf-8', low_memory=False).drop(columns=['trafficSource_campaignCode'])
testCSV = pd.read_csv('test_v2_flat.csv', sep=',', encoding='utf-8', low_memory=False)

trainCSV = fill_empty_values(trainCSV)
testCSV = fill_empty_values(testCSV)

In [ ]:
start_validation_visits = trainCSV.date.max() - timedelta(days=276)
end_validation_visits = trainCSV.date.max()

start_train_visits = start_validation_visits - timedelta(days=276)
end_train_visits = start_validation_visits

start_encoding_visits = trainCSV.date.min()
end_encoding_visits = start_train_visits

encoding_visits = restrict_df_to_time_period(trainCSV, start_encoding_visits, end_encoding_visits)
train_visits = restrict_df_to_time_period(trainCSV, start_train_visits, end_train_visits)
validation_visits = restrict_df_to_time_period(trainCSV, start_validation_visits, end_validation_visits)
test_visits =testCSV

In [ ]:
train = get_training_data_from_visits(train_visits)
validation = get_training_data_from_visits(validation_visits)
test = create_features(test_visits).reset_index()

encoder = CategoricalEncoder().fit(encoding_visits)
train_e = encoder.transform(train)
validation_e = encoder.transform(validation)
test_e = encoder.transform(test)

train_e.to_csv("train_features.csv", index=False)
validation_e.to_csv("validation_features.csv", index=False)
test_e.to_csv("test_features.csv", index=False)